# Problem: Tweet Histogram per User in 2022

**Objective:**  
Write a SQL query to obtain a histogram of tweets posted per user in 2022.  
Output the tweet count per user as the bucket and the number of Twitter users who fall into that bucket.

---

### Table: `tweets`

| Column Name | Type     |
|-------------|----------|
| tweet_id    | integer  |
| user_id     | integer  |
| msg         | string   |
| tweet_date  | timestamp|

---

### Example Input:

| tweet_id | user_id | msg                                                         | tweet_date           |
|----------|---------|-------------------------------------------------------------|----------------------|
| 214252   | 111     | Am considering taking Tesla private at $420. Funding secured.| 12/30/2021 00:00:00 |
| 739252   | 111     | Despite the constant negative press covfefe                | 01/01/2022 00:00:00 |
| 846402   | 111     | Following @NickSinghTech on Twitter changed my life!       | 02/14/2022 00:00:00 |
| 241425   | 254     | If the salary is so competitive why won’t you tell me?     | 03/01/2022 00:00:00 |
| 231574   | 148     | I no longer have a manager. I can't be managed             | 03/23/2022 00:00:00 |

---

### Expected Output:

| tweet_bucket | users_num |
|--------------|-----------|
| 1            | 2         |
| 2            | 1         |

---

### Instructions:
Write a query to:
- Filter tweets from the year 2022.
- Count how many tweets each user posted.
- Group users by the number of tweets they posted (bucket).
- Count the number of users in each bucket.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, TimestampType
from pyspark.sql.functions import *
from datetime import datetime

# Create Spark session
#spark = SparkSession.builder.master('local[*]').appName("TweetsHistogram").getOrCreate()
spark = SparkSession.builder.appName("TweetsHistogram").getOrCreate()

# Define the schema
schema = StructType([
    StructField("tweet_id", LongType(), True),
    StructField("user_id", LongType(), True),
    StructField("msg", StringType(), True),
    StructField("tweet_date", TimestampType(), True)
])

# Define the data
data = [
    (214252, 111, "Am considering taking Tesla private at $420. Funding secured.", datetime(2021, 12, 30, 0, 0)),
    (739252, 111, "Despite the constant negative press covfefe", datetime(2022, 1, 1, 0, 0)),
    (846402, 111, "Following @NickSinghTech on Twitter changed my life!", datetime(2022, 2, 14, 0, 0)),
    (241425, 254, "If the salary is so competitive why won’t you tell me what it is?", datetime(2022, 3, 1, 0, 0)),
    (231574, 148, "I no longer have a manager. I can't be managed", datetime(2022, 3, 23, 0, 0))
]

# Create the Spark DataFrame
tweets_df = spark.createDataFrame(data, schema=schema)

# Show schema and data
tweets_df.printSchema()
tweets_df.show(truncate=False)



In [0]:
#in SparkDF
tweets_df.where("tweet_date like '2022%'")\
         .groupBy('user_id').agg(count('tweet_id'))\
         .groupBy(col('count(tweet_id)').alias('tweet_bucket')).agg(count('user_id').alias('users_num'))\
         .show()

In [0]:
tweets_df.createOrReplaceTempView('tweets')

In [0]:
%%sql
select tweet_bucket, count(user_id)
from
  (SELECT count(tweet_id) as tweet_bucket, user_id FROM tweets
  WHERE EXTRACT(YEAR FROM tweet_date) = 2022
  group by user_id)x
group by tweet_bucket